In [1]:
import torch,sys,os

from tqdm import tqdm

from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

# Self reproduce

In [ ]:
# class VGG16(nn.Module):
#     def __init__(self,num_classes=102, init_weights=False):
#         super(VGG16,self).__init__()

#         self.conv1 = nn.Sequential(
#             nn.Conv2d(in_channels=3,out_channels=64,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=64,out_channels=64,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(stride=2,kernel_size=2)
#         )

#         self.conv2 = nn.Sequential(
#             nn.Conv2d(in_channels=64,out_channels=128,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=128,out_channels=128,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(stride=2,kernel_size=2)
#         )

#         self.conv3 = nn.Sequential(
#             nn.Conv2d(in_channels=128,out_channels=256,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=256,out_channels=256,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=256,out_channels=256,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(stride=2,kernel_size=2)
#         )

#         self.conv4 = nn.Sequential(
#             nn.Conv2d(in_channels=256,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(stride=2,kernel_size=2)
#         )

#         self.conv5 = nn.Sequential(
#             nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(stride=2,kernel_size=2)
#         )        

#         self.FC = nn.Sequential(
#             nn.Linear(in_features=7*7*512,out_features=4096),
#             nn.ReLU(True),
#             nn.Dropout(p=0.5),
#             nn.Linear(in_features=4096,out_features=4096),
#             nn.ReLU(True),
#             nn.Dropout(p=0.5),
#             nn.Linear(in_features=4096,out_features=num_classes)
#         )

#         if init_weights:
#             self._initialize_weights()

#     def forward(self,x):
#         x = self.conv1(x)
#         # print(x.shape)
#         x = self.conv2(x)
#         # print(x.shape)
#         x = self.conv3(x)
#         # print(x.shape)
#         x = self.conv4(x)
#         # print(x.shape)
#         x = self.conv5(x)
#         # print(x.shape)
#         x = torch.flatten(x, start_dim=1)
#         y = self.FC(x)
#         return y

#     def _initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 # nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#                 nn.init.xavier_uniform_(m.weight)
#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 nn.init.xavier_uniform_(m.weight)
#                 # nn.init.normal_(m.weight, 0, 0.01)
#                 nn.init.constant_(m.bias, 0)

# Pytorch Version

In [2]:
class VGG16(nn.Module):
    def __init__(self,num_classes=102, init_weights=False):
        super(VGG16,self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64,out_channels=64,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2,kernel_size=2),

            nn.Conv2d(in_channels=64,out_channels=128,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128,out_channels=128,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2,kernel_size=2),

            nn.Conv2d(in_channels=128,out_channels=256,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=256,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=256,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2,kernel_size=2),

            nn.Conv2d(in_channels=256,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2,kernel_size=2),

            nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512,out_channels=512,padding=1,kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2,kernel_size=2)
        )       

        self.avgpool = nn.AdaptiveAvgPool2d(output_size=7)
        

        self.Classifier = nn.Sequential(
            nn.Linear(in_features=7*7*512,out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096,out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096,out_features=num_classes)
        )

        if init_weights:
            self._initialize_weights()

    def forward(self,x):
        x = self.features(x)
        # print(x.shape)
        x = self.avgpool(x)

        x = torch.flatten(x, start_dim=1)
        y = self.Classifier(x)
        return y

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                # nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                # nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

Using cuda device


 # STL10 dataset

In [3]:
transform=transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5,),(0.5,0.5,0.5)),
	transforms.Resize([227, 227])
    ]
)

# Download training data from open datasets.
train_set = datasets.STL10(
    root="~/data/STL10/",
    split ='train',
    download=True,
    transform=transform, # transform,
)
trainloader=torch.utils.data.DataLoader(
	train_set,
	batch_size=60,
	shuffle=True,
	pin_memory=True,
    num_workers=8
	)


# Download test data from open datasets.
test_set = datasets.STL10(
    root="~/data/STL10/",
    split ='test',
    download=True,
    transform=transform, # transform,
)
testloader=torch.utils.data.DataLoader(
	test_set,
	batch_size=60,
	shuffle=False,
    pin_memory=True,
    num_workers=8
	)

# test_data_iter=iter(testloader)
# test_image,test_label=test_data_iter.next()
test_num  = len(test_set)
train_steps = len(trainloader)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


_IncompatibleKeys(missing_keys=['Classifier.0.weight', 'Classifier.0.bias', 'Classifier.3.weight', 'Classifier.3.bias', 'Classifier.6.weight', 'Classifier.6.bias'], unexpected_keys=['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])

# loss and optimize

In [5]:
# 定义一个损失函数
loss_fn = nn.CrossEntropyLoss()

# 定义一个优化器
# optimizer = torch.optim.Adam(model.parameters(),lr=0.005)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

epochs = 40

save_path= './GoogLeNet.pth'
best_acc = 0.0


# Transfer learning

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# from torchvision import models
# pretrained_googlenet = models.googlenet(pretrained=True)
# torch.save(pretrained_googlenet.state_dict(), save_path)

model = VGG16(num_classes=10, init_weights=True).to(device)
model.load_state_dict(torch.load(save_path),strict=False)

# Train

In [7]:
for epoch in range(epochs):
        # train
        model.train()
        running_loss = 0.0
        train_bar = tqdm(trainloader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = model(images.to(device))
            loss = loss_fn(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,epochs,loss)

        # validate
        model.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(testloader, file=sys.stdout) # show progress
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = model(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / test_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(model.state_dict(), save_path)

print('Finished Training')

100%|██████████| 134/134 [00:38<00:00,  3.49it/s]
[epoch 1] train_loss: 2.189  val_accuracy: 0.696
100%|██████████| 134/134 [00:40<00:00,  3.27it/s]
[epoch 2] train_loss: 1.365  val_accuracy: 0.832
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 3] train_loss: 0.854  val_accuracy: 0.872
100%|██████████| 134/134 [00:39<00:00,  3.38it/s]
[epoch 4] train_loss: 0.623  val_accuracy: 0.902
100%|██████████| 134/134 [00:41<00:00,  3.27it/s]
[epoch 5] train_loss: 0.502  val_accuracy: 0.913
100%|██████████| 134/134 [00:40<00:00,  3.30it/s]
[epoch 6] train_loss: 0.390  val_accuracy: 0.920
100%|██████████| 134/134 [00:40<00:00,  3.31it/s]
[epoch 7] train_loss: 0.339  val_accuracy: 0.924
100%|██████████| 134/134 [00:40<00:00,  3.30it/s]
[epoch 8] train_loss: 0.281  val_accuracy: 0.925
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 9] train_loss: 0.267  val_accuracy: 0.930
100%|██████████| 134/134 [00:40<00:00,  3.33it/s]
[epoch 10] train_loss: 0.233  val_accuracy: 0.933
100%|████

KeyboardInterrupt: ignored